In [1]:
# check that k3l_test contains appropirate variants for k3l
import pandas as pd
from Bio.Seq import Seq

In [16]:
# read script output
input_file = 'k3l_test.tsv'
df = pd.read_csv(input_file, sep='\t')
df.fillna('', inplace=True)

In [17]:
df.query('position == 13')

,name,sub_window_name,wt_codon,position,iupac_codon,codon_sub,iupac_aa,add_synonymous_codon,contains_missense_stop,remove_missense_stop_codon,...,sub_window,forward_primer,forward_primer_tm,forward_primer_gc,forward_primer_len,reverse_primer_name,reverse_primer,reverse_primer_tm,reverse_primer_gc,reverse_primer_len
4,window_1-1_GAT13HAT,window_1-1,GAT,13,HAT,GAT13HAT,NHY,0.0,0.0,0.0,...,gcwggwHATgtwatwaar,GGCAGAGTATACGAGAAGGATT,51.3,45.5,22,rev_window_1-1,ATTGGGCAACGAATAACAAAATGCA,55.1,36.0,25
5,window_1-1_GAT13GBT,window_1-1,GAT,13,GBT,GAT13GBT,AGV,0.0,0.0,0.0,...,gcwggwGBTgtwatwaar,GGCAGAGTATACGAGAAGGATT,51.3,45.5,22,rev_window_1-1,ATTGGGCAACGAATAACAAAATGCA,55.1,36.0,25
6,window_1-1_GAT13GAA,window_1-1,GAT,13,GAA,GAT13GAA,E,0.0,0.0,0.0,...,gcwggwGAAgtwatwaar,GGCAGAGTATACGAGAAGGATT,51.3,45.5,22,rev_window_1-1,ATTGGGCAACGAATAACAAAATGCA,55.1,36.0,25


In [18]:
# gather iupac-encoded aa missense variants by position
df1 = df.groupby('position')['iupac_aa'].apply(list).reset_index()
map_dict = dict(zip(df.position, df.wt_codon))
df1['wt_codon'] =df1.position.map(map_dict)
df1.iupac_aa = df1.iupac_aa.str.join('').str.split('')
df1.iupac_aa = df1.iupac_aa.apply(lambda x: set(x))
df1.iupac_aa.apply(lambda x: x.remove(''))
df1['wt_aa'] = df1.wt_codon.apply(lambda x: str(Seq(x).translate()))

In [19]:
# get the missense variants for the wt codon
def aa_missense_variants(codon):  
    nucleotides = 'ACGT'
    wt_aa = str(Seq(codon).translate())
    missense_aa = []
    for position in range(3):  
        for n in nucleotides:
            new_codon = codon[:position] + n + codon[position + 1:]
            new_aa = str(Seq(new_codon).translate())
            if new_aa != wt_aa:
                missense_aa.append(new_aa)
            else: 
                continue
    return set(missense_aa)
df1['wt_missense'] = df1.wt_codon.apply(aa_missense_variants)

In [20]:
# should just be stop codons removed
df1['missing_from_wt'] = df1.wt_missense - df1.iupac_aa
# should just be synonymous variants
df1['added_from_wt'] = df1.iupac_aa - df1.wt_missense
# should just be stops and wt/synonymous variants
df1['sym_diff'] = df1.apply(lambda x: x['iupac_aa'].symmetric_difference(x['wt_missense']), axis=1)

In [36]:
def check_sym_diff(row):
    sym_set = row['sym_diff']
    wt_aa = str(row['wt_aa'])
    for i in sym_set:
        if i not in ["*", wt_aa]:
            return True
    else:
        return False
        
        
df1['check_sym'] = df1.apply(check_sym_diff, axis=1)
df1.check_sym.sum()

0

In [169]:
# there's an issue with this function in codon_table module
# test with "GAT" position 13
iupac_dict = {'A':'A','C':'C','G':'G','T':'T','AC':'M','AG':'R','AT':'W','CG':'S','CT':'Y','GT':'K','ACG':'V','ACT':'H','AGT':'D','CGT':'B','ACGT':'N'}
rev_iupac_dict = {value:key for key,value in iupac_dict.items()}

def iupac_to_aa(iupac_codon):
    """Return string of AAs encoded by input iupac missense codon"""
    aa_list = []
    for i,n in enumerate(list(iupac_codon)):
        if n in list('ACGT'):
            continue
        for new_nuc in rev_iupac_dict[n]:
            new_codon = iupac_codon[:i] + new_nuc + iupac_codon[i + 1:]
            aa_list.append(str(Seq(new_codon).translate()))
    return ''.join(aa_list)

iupac_to_aa('HAT')

'NHY'

In [170]:
iupac_to_aa('GBT')

'AGV'

In [171]:
iupac_to_aa('GAA')

''

In [193]:
import itertools
a = [['A','C','T'],['A'],['T']]
[''.join(i) for i in list(itertools.product(*a))]

['AAT', 'CAT', 'TAT']

In [190]:
list(rev_iupac_dict['H'])

['A', 'C', 'T']

In [194]:
def new_iupac_to_aa(iupac_codon):
    """Return string of AAs encoded by input iupac missense codon"""
    nuc_lists = [list(rev_iupac_dict[n]) for n in iupac_codon]
    codon_list = [''.join(i) for i in list(itertools.product(*nuc_lists))]
    aa_list = [str(Seq(codon).translate()) for codon in codon_list]
    return ''.join(aa_list)

new_iupac_to_aa('HAT')

'NHY'

In [195]:
new_iupac_to_aa('GBT')

'AGV'

In [196]:
new_iupac_to_aa('GAA')

'E'

In [134]:
df1['difference_2'] = (df1.iupac_aa.apply(set) - df1.wt_aa.apply(set))
df1['contains_wt_aa'] = df1.apply(lambda x: x['wt_aa'] in x['difference'], axis=1)

df1

,position,iupac_aa,wt_codon,wt_aa,wt_missense,difference,difference_2,contains_wt_aa
0,11,"[, T, P, S, E, G, V, ]",GCG,A,"[T, P, S, E, G, V]",{A},"{, S, G, P, V, E, T}",True
1,12,"[, S, R, C, D, A, V, ]",GGT,G,"[S, R, C, D, A, V]",{G},"{, S, C, D, R, V, A}",True
2,13,"[, N, H, Y, A, G, V, ]",GAT,D,"[N, H, Y, A, G, V, E, E]",{D},"{, G, N, V, A, Y, H}",True
3,14,"[, I, L, E, A, G, ]",GTA,V,"[I, L, L, E, A, G]",{V},"{, G, I, L, A, E}",True
4,15,"[, V, L, K, T, R, ]",ATA,I,"[L, V, L, K, T, R, M]",{I},"{, K, V, R, L, T}",True
5,16,"[, Q, E, *, T, R, M, K, N, ]",AAG,K,"[Q, E, *, T, R, M, N, N]",{},"{, Q, N, R, *, E, M, T}",False
6,17,"[, S, R, C, D, A, V, ]",GGC,G,"[S, R, C, D, A, V]",{G},"{, S, C, D, R, V, A}",True
7,18,"[, G, *, K, T, I, ]",AGA,R,"[G, *, K, T, I, S, S]",{R},"{, G, I, K, *, T}",True
8,43,"[, G, C, N, T, I, ]",AGT,S,"[R, G, C, N, T, I, R, R]",{S},"{, G, I, N, C, T}",True
9,44,"[, I, L, F, D, A, G, ]",GTT,V,"[I, L, F, D, A, G]",{V},"{, G, I, D, L, A, F}",True


In [138]:
a = {'', 'a', 't'}
a.remove('')
a


{'a', 't'}

0           {E, G, S, T, P, V}
1           {S, C, V, D, R, A}
2           {G, N, V, Y, H, A}
3              {E, G, I, L, A}
4              {K, T, V, R, L}
5     {Q, N, K, R, *, E, M, T}
6           {S, C, V, D, R, A}
7              {G, I, K, T, *}
8              {G, I, N, T, C}
9           {G, F, I, D, L, A}
10          {E, Q, M, T, R, *}
11             {K, T, V, R, L}
12          {N, P, D, R, L, Y}
13             {K, T, V, R, L}
14          {G, N, V, Y, H, A}
15             {G, I, K, T, *}
16          {S, F, N, C, D, H}
17          {G, N, V, Y, H, A}
18          {S, F, N, C, D, H}
19          {S, I, K, P, R, A}
20          {E, Q, I, T, R, *}
21             {E, V, R, *, A}
22          {S, F, N, C, D, H}
23             {K, T, V, R, L}
24          {G, N, V, Y, H, A}
25          {G, F, I, D, L, A}
26          {S, I, T, D, H, Y}
27          {S, F, N, C, D, H}
28          {E, Q, I, T, R, *}
29       {G, W, S, M, K, T, R}
30             {K, T, V, R, L}
31       {G, S, W, F, Y, R, *}
32      

In [135]:
df1.difference == df1.difference_2

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
dtype: bool

In [5]:
# check that synonymous variants look correct
Seq('HCGggwgaygtwatwaar').translate()

Seq('XGDVIK')

In [32]:
# add the aa info to the iupacs
df=pd.read_csv('data/final_codon_table.csv')
df.fillna('', inplace=True)
df.head()

,codon,aa,position,missense_nuc,missense_codons,missense_aa,missense_iupac,missense_iupac_codon,sele_codons,sele_aa,...,syn_bool,syn_codons,syn_aa,syn_iupac_codon,no_stop_codons,no_stop_aa,no_stop_iupac_codon,no_stop_syn_codons,no_stop_syn_aa,no_stop_syn_iupac_codon
0,AAA,K,0,CGT,CAA GAA TAA,*QE,B,BAA,CAA GAA TAA,QE*,...,False,,,BAA,CAA GAA,QE,SAA,CAA GAA,QE,SAA
1,AAA,K,1,CGT,ACA AGA ATA,TRI,B,ABA,ACA AGA ATA,TRI,...,False,,,ABA,ACA AGA ATA,TRI,ABA,ACA AGA ATA,TRI,ABA
2,AAA,K,2,CT,AAC AAT,N,Y,AAY,AAT,N,...,True,AAG AAT,KN,AAK,AAT,N,AAT,AAG AAT,KN,AAK
3,AAC,N,0,CGT,CAC GAC TAC,HDY,B,BAC,CAC GAC TAC,HDY,...,False,,,BAC,CAC GAC TAC,HDY,BAC,CAC GAC TAC,HDY,BAC
4,AAC,N,1,CGT,ACC AGC ATC,TSI,B,ABC,ACC AGC ATC,TSI,...,False,,,ABC,ACC AGC ATC,TSI,ABC,ACC AGC ATC,TSI,ABC


In [50]:
# function that translates iupac codon into all AAs
iupac_dict = {'A':'A','C':'C','G':'G','T':'T','AC':'M','AG':'R','AT':'W','CG':'S','CT':'Y','GT':'K','ACG':'V','ACT':'H','AGT':'D','CGT':'B','ACGT':'N'}
rev_iupac_dict = {value:key for key,value in iupac_dict.items()}


def iupac_to_aa(iupac_codon):
    """Return string of AAs encoded by input iupac missense codon"""
    aa_list = []
    for i,n in enumerate(list(codon)):
        if n in list('ACGT'):
            continue
        for new_nuc in rev_iupac_dict[n]:
            new_codon = codon[:i] + new_nuc + codon[i + 1:]
            aa_list.append(str(Seq(new_codon).translate()))
    return ''.join(aa_list)

iupac_to_aa('BAA')

'QE*'

In [48]:
aas

['Q', 'E', '*']

In [34]:
test_dict

{'AAA': {'iupac': ['BAA', 'ABA', 'AAT']},
 'AAC': {'iupac': ['BAC', 'ABC', 'AAA']},
 'AAG': {'iupac': ['BAG', 'ABG', 'AAT']},
 'AAT': {'iupac': ['BAT', 'ABT', 'AAA']},
 'ACA': {'iupac': ['BCA', 'ADA']},
 'ACC': {'iupac': ['BCC', 'AWC']},
 'ACG': {'iupac': ['BCG', 'ADG']},
 'ACT': {'iupac': ['BCT', 'AWT']},
 'AGA': {'iupac': ['KGA', 'AHA', 'AGT']},
 'AGC': {'iupac': ['KGC', 'AHC', 'AGA']},
 'AGG': {'iupac': ['KGG', 'AHG', 'AGT']},
 'AGT': {'iupac': ['KGT', 'AHT', 'AGA']},
 'ATA': {'iupac': ['KTA', 'AVA', 'ATG']},
 'ATC': {'iupac': ['BTC', 'AVC', 'ATG']},
 'ATG': {'iupac': ['KTG', 'AVG', 'ATT']},
 'ATT': {'iupac': ['BTT', 'AVT', 'ATG']},
 'CAA': {'iupac': ['DAA', 'CBA', 'CAT']},
 'CAC': {'iupac': ['DAC', 'CBC', 'CAA']},
 'CAG': {'iupac': ['DAG', 'CBG', 'CAT']},
 'CAT': {'iupac': ['DAT', 'CBT', 'CAA']},
 'CCA': {'iupac': ['DCA', 'CDA']},
 'CCC': {'iupac': ['DCC', 'CDC']},
 'CCG': {'iupac': ['DCG', 'CDG']},
 'CCT': {'iupac': ['DCT', 'CDT']},
 'CGA': {'iupac': ['KGA', 'CHA']},
 'CGC': {'iup

In [ ]:
for key,value in temp_dict.items():
    temp_dict[key] = list(itertools.chain.from_iterable([codon.split(' ') for codon in value]))

In [28]:
df.groupby('codon')[['sele_iupac_codon','sele_aa']].apply(lambda x: x.to_dict(orient='index')).to_dict()

{'AAA': {0: {'sele_iupac_codon': 'BAA', 'sele_aa': 'QE*'},
  1: {'sele_iupac_codon': 'ABA', 'sele_aa': 'TRI'},
  2: {'sele_iupac_codon': 'AAT', 'sele_aa': 'N'}},
 'AAC': {3: {'sele_iupac_codon': 'BAC', 'sele_aa': 'HDY'},
  4: {'sele_iupac_codon': 'ABC', 'sele_aa': 'TSI'},
  5: {'sele_iupac_codon': 'AAA', 'sele_aa': 'K'}},
 'AAG': {6: {'sele_iupac_codon': 'BAG', 'sele_aa': 'QE*'},
  7: {'sele_iupac_codon': 'ABG', 'sele_aa': 'TRM'},
  8: {'sele_iupac_codon': 'AAT', 'sele_aa': 'N'}},
 'AAT': {9: {'sele_iupac_codon': 'BAT', 'sele_aa': 'HDY'},
  10: {'sele_iupac_codon': 'ABT', 'sele_aa': 'TSI'},
  11: {'sele_iupac_codon': 'AAA', 'sele_aa': 'K'}},
 'ACA': {12: {'sele_iupac_codon': 'BCA', 'sele_aa': 'PAS'},
  13: {'sele_iupac_codon': 'ADA', 'sele_aa': 'KRI'},
  14: {'sele_iupac_codon': nan, 'sele_aa': nan}},
 'ACC': {15: {'sele_iupac_codon': 'BCC', 'sele_aa': 'PAS'},
  16: {'sele_iupac_codon': 'AWC', 'sele_aa': 'NI'},
  17: {'sele_iupac_codon': nan, 'sele_aa': nan}},
 'ACG': {18: {'sele_iupac

In [23]:
# check that iupac encodes all AAs for given position

iupac_dict = {'A':'A','C':'C','G':'G','T':'T','AC':'M','AG':'R','AT':'W','CG':'S','CT':'Y','GT':'K','ACG':'V','ACT':'H','AGT':'D','CGT':'B','ACGT':'N'}
rev_iupac_dict = {value:key for key,value in iupac_dict.items()}

df1 = df.groupby('position')['iupac'].apply(list).reset_index()
map_dict = dict(zip(df.position, df.wt))
df1['wt'] =df1.position.map(map_dict)
df1.head()

,position,iupac,wt
0,11,"[HCG, GDG]",GCG
1,12,"[HGT, GHT]",GGT
2,13,"[HAT, GBT, GAA]",GAT
3,14,"[WTA, GVA]",GTA
4,15,"[KTA, AVA, ATG]",ATA


In [ ]:
# get iupac aas
def iupac_aas(row):
    iupac_codons = row['iupac']
    for iupac in iupac_codons

In [20]:
map_dict = dict(zip(df.position, df.wt))

{11: 'GCG',
 12: 'GGT',
 13: 'GAT',
 14: 'GTA',
 15: 'ATA',
 16: 'AAG',
 17: 'GGC',
 18: 'AGA',
 43: 'AGT',
 44: 'GTT',
 45: 'AAG',
 46: 'ATG',
 47: 'CAT',
 48: 'ATG',
 49: 'GAT',
 50: 'AGA',
 51: 'TAT',
 71: 'GAT',
 72: 'TAT',
 73: 'ACA',
 74: 'AAA',
 75: 'GGA',
 76: 'TAT',
 77: 'ATA',
 78: 'GAT',
 79: 'GTC',
 80: 'AAT',
 81: 'TAC',
 82: 'AAA',
 83: 'AGG',
 84: 'ATG',
 85: 'TGT',
 86: 'AGA',
 87: 'CAT',
 88: 'CAA'}